In [33]:
from sklearn.datasets import fetch_openml
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

dataset = fetch_openml("diabetes", version=1, as_frame=True, return_X_y=False)["frame"]

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


import all required dependencies and load in the dataset

In [17]:
print(type(dataset))
print(dataset.head())

<class 'pandas.core.frame.DataFrame'>
   preg   plas  pres  skin   insu  mass   pedi   age            class
0   6.0  148.0  72.0  35.0    0.0  33.6  0.627  50.0  tested_positive
1   1.0   85.0  66.0  29.0    0.0  26.6  0.351  31.0  tested_negative
2   8.0  183.0  64.0   0.0    0.0  23.3  0.672  32.0  tested_positive
3   1.0   89.0  66.0  23.0   94.0  28.1  0.167  21.0  tested_negative
4   0.0  137.0  40.0  35.0  168.0  43.1  2.288  33.0  tested_positive


In [18]:
import sqlite3

connection = sqlite3.connect("diabetes.db")
cursor = connection.cursor()

establish a SQL connection and create a new diabetes database

In [19]:
cursor.execute('DROP TABLE IF EXISTS diabetes')
create_sql_query = """
    CREATE TABLE diabetes(
        preg NUM,
        plas NUM,
        pres NUM,
        skin NUM,
        insu NUM,
        mass NUM,
        pedi NUM,
        age NUM,
        class TEXT
    )
"""
cursor.execute(create_sql_query)




create a table with the specified features which will be
used to train a classifyer

In [20]:
insert_sql_query = "INSERT INTO diabetes VALUES (?,?,?,?,?,?,?,?,?)"
data_rows = dataset.to_numpy().tolist()
cursor.executemany(insert_sql_query, data_rows)


repeatedly executes a SQL query that fills in the columns of the diabetes table of the database with the values extracted from the loaded dataset

In [21]:
cursor.commit()
cursor.close()

close the connection with the server after inputting all the values into the diabetes table

In [28]:
def data_generator(batch_size):

    connection = sqlite3.connect("diabetes.db", check_same_thread=False)
    cursor = connection.cursor()
    sql_query = f"""
        SELECT preg, plas, pres, skin, insu, mass, pedi, age, class
        FROM diabetes
        ORDER BY random()
        LIMIT {batch_size}
    """
    while True:
        cursor.execute(sql_query)
        ran_sample = cursor.fetchall()
        X_train = [row[:-1] for row in ran_sample]
        y_train = [1 if row[-1]=="tested_positive" else 0 for row in ran_sample]
        yield np.asarray(X_train), np.asarray(y_train)




create a function that connects to the database and retrieves a random sample with a specified batch size. This labeled data will be used to train the classifying neural network.

In [23]:
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

creates a neural network with 3 layers and the third being the output layer.

In [29]:
history = model.fit(data_generator(300), epochs=5, steps_per_epoch=2000)


Epoch 1/5
2000/2000 [==============================] - 11s 5ms/step - loss: 0.4296 - accuracy: 0.7944
Epoch 2/5
2000/2000 [==============================] - 10s 5ms/step - loss: 0.4118 - accuracy: 0.8046
Epoch 3/5
2000/2000 [==============================] - 11s 5ms/step - loss: 0.3983 - accuracy: 0.8099
Epoch 4/5
2000/2000 [==============================] - 12s 6ms/step - loss: 0.3918 - accuracy: 0.8118
Epoch 5/5
2000/2000 [==============================] - 11s 5ms/step - loss: 0.3840 - accuracy: 0.8158


fit the model with the training data; in this case, we used 300 training examples.

In [42]:
def testing_data_generator(batch_size):

    connection = sqlite3.connect("diabetes.db", check_same_thread=False)
    cursor = connection.cursor()
    sql_query = f"""
        SELECT preg, plas, pres, skin, insu, mass, pedi, age, class
        FROM diabetes
        ORDER BY random()
        LIMIT {batch_size}
    """
    cursor.execute(sql_query)
    ran_sample = cursor.fetchall()
    X_test = [row[:-1] for row in ran_sample]
    y_test = [1 if row[-1]=="tested_positive" else 0 for row in ran_sample]


    return (np.asarray(y_test), model.predict(np.asarray(X_test)))

tup = testing_data_generator(600)

labels = tup[0]
predictions = tup[1]

confusion_matrix = tf.math.confusion_matrix(labels, predictions)

print(confusion_matrix)



19/19 [==============================] - 0s 3ms/step
tf.Tensor(
[[389   0]
 [205   6]], shape=(2, 2), dtype=int32)


created a function to generate testing data. Used this function to test the model on 600 rows of data. Printed out the confusion matrix to show how the model performed.